In [ ]:
!pip uninstall urllib3 -y
!pip install --upgrade urllib3
!pip install --upgrade ai2thor ai2thor-colab &> /dev/null
import ai2thor
import ai2thor_colab

from ai2thor.controller import Controller
from ai2thor_colab import (
    plot_frames,
    show_objects_table,
    side_by_side,
    overlay,
    show_video
)

ai2thor_colab.start_xserver()

"AI2-THOR Version: " + ai2thor.__version__

Found existing installation: urllib3 1.26.14
Uninstalling urllib3-1.26.14:
  Successfully uninstalled urllib3-1.26.14
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.6 MB/s eta 0:00:00


'AI2-THOR Version: 5.0.0'

In [ ]:
#!pip install torch==1.12.0
!pip install transformers
#!pip install torchvision==0.13.0
!pip install gym
!pip install stable_baselines3
!pip install einops
!pip install img2vec_pytorch
!pip install sb3_contrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 125.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616823 sha256=8c8d5e7afbead1b624a758ce191c4bc742a83b956598f40d1d08ea698faca02d
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successf

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import resnet50, ResNet50_Weights
from transformers import AutoTokenizer, AutoModel

weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# CGAN Generator Model
class FFTranslator(nn.Module):
    def __init__(self) -> None:
        super(FFTranslator, self).__init__()
        
        self.linear1 = nn.Linear(512, 768)
        self.ln1 = nn.LayerNorm(768)
        self.linear2 = nn.Linear(768, 1024)
        self.ln2 = nn.LayerNorm(1024)
        self.linear3 = nn.Linear(1024, 1536)
        self.ln3 = nn.LayerNorm(1536)
        self.linear4 = nn.Linear(1536, 2048)
    
    def forward(self, input):
        out = self.ln1(F.relu(self.linear1(input)))
        out = self.ln2(F.relu(self.linear2(out)))
        out = self.ln3(F.relu(self.linear3(out)))
        out = torch.tanh(self.linear4(out))
        return out

In [ ]:
class GeneratorPreprocess(nn.Module):
      def __init__(self):
          super(GeneratorPreprocess, self).__init__()
          self.tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
          self.model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
          
      @torch.no_grad()
      def forward(self, text):
          encoded_input = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')#.to(device=torch.device("cuda:0"))
          out = self.model(**encoded_input)
          out = mean_pooling(out, encoded_input["attention_mask"])
          out = F.normalize(out)
          return out

In [ ]:
class Generator(nn.Module):
    def __init__(self) -> None:
        super(Generator, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
        self.model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
        
        self.translator = FFTranslator()

        # hyperparams and train params

    def forward(self, input):
        noise = self.generate_noise()#.to(device=torch.device("cuda:0"))
        out = torch.cat((torch.squeeze(input), torch.squeeze(noise)), dim=0)
        out = self.translator(out)
        return out

    def generate_noise(self):
        return torch.normal(mean=torch.zeros(128), std=torch.zeros(128))

In [ ]:
# embedder
import stable_baselines3
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from torchvision import transforms
import gym
from einops import rearrange
from PIL import Image
from img2vec_pytorch import Img2Vec

device = torch.device("cuda:0")

if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
    
device = torch.device(dev)

# model = model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.nn.Sequential(*(list(model.children())[:-1]))

# feature_extractor = model

class FeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 512):
        super(FeatureExtractor, self).__init__(observation_space, features_dim)
        
        # networks
        self.generator = Generator()
        self.generator.load_state_dict(torch.load('./generator (1).pt', map_location="cpu"))
        # self.feature_extractor = feature_extractor
        # self.old_preprocess = weights.transforms()
        # self.preprocess = transforms.Compose([
        #     transforms.ToPILImage(),
        #     transforms.Resize(256),
        #     transforms.CenterCrop(224),
        #     transforms.ToTensor(),
        #     transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
        # ])

        
        # load torch parameters for generator
        
        self.concat_processor = nn.Sequential(
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, features_dim),
            nn.Sigmoid()
        )
        
    def forward(self, obs_space: gym.spaces.Box):
        image = torch.empty((obs_space.size()[0], 2048))
        text = torch.empty((obs_space.size()[0], 384))
        for ix in range(obs_space.size()[0]):
            image[ix] = obs_space[ix,:-384,0]
            text[ix] = obs_space[ix,2048:2048+384,0]
        out = torch.empty(image.size(dim=0), 512)
        for ix in range(image.size(dim=0)):
          with torch.no_grad():
              #filtered_input = rearrange(image[ix].squeeze(), 'h w c -> c h w')
              #image_embed = self.preprocess(filtered_input)
              # = torch.squeeze(self.feature_extractor(image_embed.unsqueeze(0)))
              text_embed = torch.squeeze(self.generator(text[ix].squeeze()))
          concat_embed = torch.cat((image[ix].squeeze(), F.normalize(text_embed, dim=0)), dim=0)
          out[ix] = self.concat_processor(concat_embed)
        return out

def execute_resnet(image):
  with torch.no_grad():
    tensor = torch.Tensor(image.copy())
    fe = Img2Vec(cuda=False, model='resnet50')
    # preprocess = transforms.Compose([
    #     transforms.RandomResizedCrop(224),
    #     transforms.RandomHorizontalFlip(),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # ])
    tpi = transforms.ToPILImage()
    reordered = tpi(rearrange(tensor, 'h w c -> c h w'))
    image_embed = torch.squeeze(torch.from_numpy(fe.get_vec(reordered)))
  return F.normalize(image_embed, dim=0)

In [ ]:
kitchens = [f"FloorPlan{i}" for i in range(1, 21)]
living_rooms = [f"FloorPlan{200 + i}" for i in range(1, 21)]
bedrooms = [f"FloorPlan{300 + i}" for i in range(1, 21)]
bathrooms = [f"FloorPlan{400 + i}" for i in range(1, 21)]

trainMats = kitchens + living_rooms + bedrooms + bathrooms

kitchens = [f"FloorPlan{i}" for i in range(21, 26)]
living_rooms = [f"FloorPlan{200 + i}" for i in range(21, 26)]
bedrooms = [f"FloorPlan{300 + i}" for i in range(21, 26)]
bathrooms = [f"FloorPlan{400 + i}" for i in range(21, 26)]

valMats = kitchens + living_rooms + bedrooms + bathrooms

kitchens = [f"FloorPlan{i}" for i in range(26, 31)]
living_rooms = [f"FloorPlan{200 + i}" for i in range(26, 31)]
bedrooms = [f"FloorPlan{300 + i}" for i in range(26, 31)]
bathrooms = [f"FloorPlan{400 + i}" for i in range(26, 31)]

testMats = kitchens + living_rooms + bedrooms + bathrooms

In [ ]:
objects = {
    "AlarmClock": "An alarm clock is a small black box with two numbers separated by a colon on a digital display in the front.",
    "AluminumFoil": "Aluminum foil is a shiny gray sheet that has a wrinkled metallic look.",
    "Apple": "An apple is a red spherical shaped fruit with a small stem on top.",
    "ArmChair": "An armchair is a large object with a lowered center to sit on and hightened sides and back to lay your arms and back on.",
    "BaseballBat": "A baseball bat is a wooden elongated stick with a circular shape and a handle and widened part on opposite ends.",
    "BasketBall": "A basketball is an orange color ball with bumps on it and black lines across it.",
    "Bathtub": "A bathtub is a large white container with a metallic tap on one end to take baths in.",
    "BathtubBasin": "A bathtub basin is a large white container with a metallic tap on one end to take baths in.",
    "Bed": "A bed is a large white mat on top of a wooden frame that is used to sleep in.",
    "Blinds": "Blinds are a set of thin wooden bars on windows that prevents you from seeing outside.",
    "Book": "A book is a medium-sized rectangular item that has multiple pages when opened, and has a hard cover.",
    "Boots": "Boots are a pair of medium-size shoes that are tall",
    "Bowl": "A bowl is a wooden container with a circular bottom to hold items, usually food",
    "Box": "A box is a cube-shaped brown item that can be used to store other items in when opened",
    "Bread": "Bread is an edible item that is brown with a lighter inside, which usually has holes",
    "ButterKnife": "A butter knife is a thin metallic object with a sharp edge used to cut butter when picked up",
    "Cabinet": "A cabinet is a brown, wooden container connected to the wall that can store items when opened",
    "Candle": "A candle is a small white stick of wax that can be lit with fire to provide light",
    "CD": "A CD is a shiny, circular object with a hole in the middle used to play videos",
    "CellPhone": "A cellphone is a small rectangular object with a translucent screen on the front and buttons on the sides",
    "Chair": "A chair is an L-shaped wooden item that can be sat on, but does not have raised arm sides",
    "Cloth": "A cloth is a flexible item found on tables that can be used to clean up areas",
    "CoffeeMachine": "A coffee machine is a black mechanism with a stand to put a cup on where a dispenser is",
    "CounterTop": "A countertop is a white flat surface elevated off the ground that is thick and patterned with abstract black spots.",
    "CreditCard": "A credit card is a small, thin rectangular object with rounded corners and text on it",
    "Cup": "A cup is a small cylinder with a hollow top to fill with items or liquids",
    "Curtains": "Curtains are hanging cloths usually found on windows to cover from what is outside it",
    "Desk": "A desk is a short table with a wooden top that usually comes with a chair",
    "DeskLamp": "A desk lamp is a source of light that can be angled and placed on a desk",
    "Desktop": "A desktop consists of a large metal box and a corresponding flat screen only a few feet wide",
    "DiningTable": "A dining table is a wooden table with 4 legs and supports, usually found at the center of a kitchen or living room",
    "DishSponge": "A dish sponge is asmall yellow rectangular item that is found near a kitchen sink for cleaning",
    "DogBed": "A dog bed is circular elevated area found in living rooms or bedrooms in corners",
    "Drawer": "A drawer is a wooden container that can be pulled horizontally to open and store things in that is usually found on the walls of kitchens or on dressers.",
    "Dresser": "A dresser is a large wooden object that consists of many drawers for storage, usually found near the walls of a room.",
    "Dumbbell": "A dumbbell is a metal stick with two black cylindrical weights on either side, usually found in corners of a room",
    "Egg": "An egg is a white oval-shaped item that can be cracked and is usually found on kitchen counters",
    "Faucet": "A faucet is a metal fixture on sinks in bathrooms or bedrooms where water comes from",
    "Floor": "The floor is the structure below you in every room that supports you",
    "FloorLamp": "A floor lamp is a tall light fixture that has a base on the floor to support it",
    "Footstool": "A footstool is a small object that looks like a miniature table that can be stepped on to reach higher elevations",
    "Fork": "A fork is a small metal utensil with 3 prongs at the end to hold onto food when eating on tables",
    "Fridge": "A fridge is a large white structure next to walls in kitchens that holds food at cold temperatures",
    "GarbageBag": "A garbage bag is a black flimsy object with wrinkles and an open top to place things in",
    "GarbageCan": "A garbage can is a tall gray cylindrical object with an open top, usually found in the corners of rooms",
    "HandTowel": "A hand towel is a small white square of cloth usually found on a holder near a sink",
    "Kettle": "A kettle is a black spherical object with a handle at the top and a shaft protruding from it to pour liquid out of",
    "KeyChain": "A key chain is a metallic circle usually found on tables with many metal or plastic keys or other objects dangling from it",
    "Knife": "A knife is a small, thin, metal utensil usually found on dining tables or in kitchens; it has a sharp edge on one side",
    "Ladle": "A ladle is a metal utensil with a partial sphere on one end to scoop items and a long handle on the other end to hold onto",
    "Laptop": "A laptop is a rectangular metal object that can be opened to reveal a screen within, usually found on tables",
    "Lettuce": "Lettuce is a small green spherical vegetable found on tables in kitchens",
    "LightSwitch": "A light switch is a small rectangular area on walls next to doors with a switch in the middle",
    "Microwave": "A microwave is a cubical metallic object with a clear front found on kitchen counters",
    "Mirror": "A mirror is a fixture on walls or a handheld item that reflects the image that faces it",
    "Mug": "A mug is a small cylindrical object with a handle at its side, usually placed on tables or filled with liquid",
    "Newspaper": "A newspaper is a rectangular stack of paper with text written on it, usually found on tables",
    "Ottoman": "An ottoman is a stool situated next to sofas or chairs made of soft materials",
    "Painting": "A painting is an intricate two dimensional rectangle of colors placed on a wall",
    "Pan": "A pan is a black circular object with small walls and a handle protruding from it, usually found on kitchen counters or stoves",
    "PaperTowelRoll": "A paper towel roll is a metal fixture with a vertical roll of white paper wrapped around it, usually found on kitchen counters",
    "Pen": "A pen is a long black stick with a sharp point at one end, usually found on tables",
    "Pencil": "A pencil is a yellow wooden stick with a sharp point at one end, usually found on tables",
    "PepperShaker": "A pepper shaker is a cylindrical object filled with black powder in the middle, topped with a metallic cap",
    "Pillow": "A pillow is a soft white rectangle found at the front of beds",
    "Plate": "A plate is a circular, smooth, white object with an incline at the ends, usually found on tables or kitchen counters",
    "Plunger": "A plunger is a long stick with a half-circle attached to it at the end so that the half-circle is facing down",
    "Poster": "A poster is a large picture of anything pasted onto a wall",
    "Pot": "A pot is a black circular object with tall walls and a handle protruding from it, usually found on kitchen counters or stoves",
    "Potato": "A potato is a spherical brown object with dots on it, usually found on kitchen counters",
    "RemoteControl": "A remote control is a thin black rectangular object with assorted buttons on it, usually found on a table near a sofa",
    "Safe": "A safe is a gray metal box usually placed on tables that can be opened with a lock password.",
    "SaltShaker": "A salt shaker is a clear cylindrical container filled with white powder topped with a metal cap with holes.",
    "ScrubBrush": "A scrub brush is a wooden shaft with a handle on one end and bristles on the other end to brush hair, and it is usually found on tables.",
    "Shelf": "A wooden plank situated on a wall to place items on at an elevated height",
    "ShelvingUnit": "A combination of multiple shelves to store items at different levels of elevation along walls",
    "ShowerCurtain": "A vertical cloth that covers the front of a shower in bathrooms",
    "ShowerDoor": "A clear door that covers showers in bathrooms to prevent water from falling out",
    "ShowerGlass": "Shower glass covers the front of showers in a clear material in bathrooms",
    "ShowerHead": "A shower head is a metal object in a shower where water comes from",
    "SideTable": "A side table small table found next to sofas, beds, or chairs that is usually elevated and made of wood",
    "Sink": "A sink is a cube-shaped object with a hole at the top found in kitchens and bathrooms with a faucet to wash hands and dishes",
    "SoapBar": "A soap bar is a small white rectangle found in the showers of bathrooms that is used for bathing",
    "SoapBottle": "A soap bottle is a cylindrical object found in showers that contains liquid for cleaning",
    "Sofa": "A sofa is a large rectangular object made of soft materials in centers of living rooms for sitting on",
    "Spatula": "A spatula is a metal utensil with a handle on one side and a widened flat part on the other side for picking up and lifting objects during cooking.",
    "Spoon": "A spoon is a small  metal utensil with a handle on one side and a head with an inside curve to scoop up items while eating.",
    "SprayBottle": "A spray bottle is a cylindrical translucent container with a hook-shaped head to spray liquids within the container, usually found near cabinets.",
    "Statue": "A statue is an abstract 3d object made of stone usually found on tables that is used as a decorative item.",
    "Stool": "A stool is a small table usually placed on the floor that people can stand on to increase height.",
    "StoveBurner": "A stove burner is a large metal box with a black top situated along the walls of kitchens between counters.",
    "StoveKnob": "A stove knob is a small circular metal item connected to a stove burner that can be turned.",
    "TableTopDecor": "Table top decor is a small decorative item found on top of tables",
    "TeddyBear": "A teddy bear is a brown, fluffy item in the shape of an animal usually found on beds or nightstands.",
    "Television": "A television is a large, black, flat screen situated near walls that can be turned on to view different things.",
    "TennisRacket": "A tennis racket is an ovular shape with intersecting lines of rope in the middle, as well as a handle on one end.",
    "TissueBox": "A tissue box is a rectangular shape object with an opening at the top to pull tissues, or small white squares, out of.",
    "Toaster": "A toaster is a metal cube-shaped objects with two to four slits at the top to insert bread in to heat it.",
    "Toilet": "A toilet is a white ceramic object found in bathrooms that connects directly to the wall and has an ovular opening in the middle.",
    "ToiletPaper": "Toilet paper consists of a roll of white paper situated near a toilet in bathrooms",
    "ToiletPaperHanger": "A toilet paper hanger is a metal L-shaped item protruding from the wall of of bathroom near a toilet",
    "Tomato": "A tomato is a red circular object with a green stem on top, usually found on kitchen counters",
    "Towel": "A towel is a white rectangular object made of cloth that hangs from walls in bathrooms.",
    "TowelHolder": "A towel holder is a metal bar hanging from the walls of a bathroom that holds a white towel.",
    "TVStand": "A television stand is a black metal bracket under a television that holds it up from the floor.",
    "VacuumCleaner": "A vacuum cleaner is a long machine with wheels under and a handle at the top and a dust container in the middle.",
    "Vase": "A vase is an intricately shaped cylindrical object with an open top to place flowers or liquids within",
    "Watch": "A small circular object with a white face surrounded by a a border and two metal bands coming off of it.",
    "WateringCan": "A watering can is a plastic cylinder with a handle on top and a small pipe to pour water from.",
    "Window": "A window is a clear rectangular space on a wall that can be seen through"
}

In [ ]:
from ai2thor.controller import Controller
import gym
import torch
import gym.spaces as spaces
import random
from stable_baselines3.common.policies import obs_as_tensor
from collections import OrderedDict
import cv2

def predict_proba(model, state, lstm):
    obs = obs_as_tensor(state.copy(), model.policy.device)
    obs = obs.unsqueeze(0)
    episode_starts = torch.ones(1)
    # obs = obs.unsqueeze(2)
    print(obs.size())
    dis = model.policy.get_distribution(obs, lstm, episode_starts)
    #probs = dis.distribution.probs
    #probs_np = probs.detach().numpy()
    return dis


class AI2thor(gym.Env):
    def __init__(self, ttv):
        super(AI2thor, self).__init__()
        
        if ttv=='train':
            self.rooms = trainMats
        elif ttv=='val':
            self.rooms = valMats
        else:
            self.rooms = testMats
            
        self.old_actions = ["MoveAhead", "MoveBack", "MoveLeft", "MoveRight", "RotateLeft", "RotateRight", "LookUp", "LookDown", "Crouch", "Stand", "Done"]
        self.actions = ["MoveAhead", "MoveBack", "MoveLeft", "MoveRight", "RotateLeft", "RotateRight", "Done"]
        self.action_space = spaces.Discrete(7)
        
        # self.observation_space = spaces.Dict({
        #     "image": spaces.Box(low=0, high=255, shape=(448, 448, 3)),
        #     "text": spaces.Box(low=-1.0, high=1.0, shape=(1,384))
        # })
        self.observation_space = spaces.Box(low=-1.0, high=255, shape=(2048+384, 1))
        
        self.text_processor = GeneratorPreprocess()
            
        self.ttv = ttv
        self.controller = None
        self.current_obj = None
        self.obj_position = None
        self.done = False
        self.obj_vect = None
        self.step_count = 0
        self.total_r = 0;
        self.total_rs = []
        self.chosen_objects = []
        self.num_objs = 0
                
    def step(self, action):
        print("step: ", self.step_count)
        print("action: ", self.actions[action])
        event = self.controller.step(self.actions[action])
        reward = 0
        self.total_r+=reward
        done = False
        if self.actions[action]=="Done" or self.step_count>=32:
                ag_pos = event.metadata["agent"]["position"]
                reward = 1-0.25*self.compute_distance(ag_pos)
                if reward<0: reward=0
                self.total_r+=reward
                print("Finished! End reward: ",self.total_r)
                self.total_rs.append(self.total_r)   
                done = True
                print("\n")
        info = {
            "episode": None,
            "is_success": None
        }
        #probs = predict_proba(model, self.current_state)
        #print("probs: ",probs)
        cv2.imwrite("./im_{}.jpg".format(self.step_count), cv2.cvtColor(event.cv2img, cv2.COLOR_RGB2BGR))
        self.current_state = self.observation_space.sample()
        #print("actionsuccess: ", event.metadata["lastActionSuccess"])
        print("error: ", event.metadata["errorMessage"])
        # self.current_state["image"] = event.frame
        # self.current_state["text"] = self.obj_vect.numpy()
        self.current_state = self.combine_inputs(event.frame.astype(np.uint8), self.obj_vect.detach().numpy())
        self.step_count+=1;

        return self.current_state, reward, done, info
        

    def reset(self):
        self.step_count = 0
        self.total_r = 0
        self.controller = Controller(height=448, width=448, gridSize=0.25, snapToGrid=False)
        self.current_obj = None
        self.done = False
        self.current_state = self.observation_space.sample()

        #if self.num_objs>=3 or self.num_objs==0:
        self.num_objs = 0
        self.chosen_objects.clear()
        self.choose_scene()

        event, obj_vect = self.domain_rand()
        # self.current_state["image"] = event.frame
        # self.current_state["text"] = obj_vect.numpy()
        self.current_state = self.combine_inputs(event.frame.astype(np.uint8), obj_vect.detach().numpy())
        print("reset")
        self.num_objs += 1
        return self.current_state

    def combine_inputs(self, frame, obj_vect):
        ar = np.zeros(2048+384)
        ar[:-384] = execute_resnet(frame).numpy()
        ar[2048:2048+384] = obj_vect
        return np.expand_dims(ar, axis=1)
        
    def choose_scene(self):
        roomID = self.rooms[np.random.randint(0, len(self.rooms))]
        self.controller.reset(scene=roomID)

    def reset_current_scene(self):
        metadata = self.controller.last_event.metadata["sceneBounds"]
        x = set()
        y = set()
        for m in metadata:
          x.add(m[0])
          y.add(m[1])
        random_x = random.uniform(x[0], x[1])
        random_y = random.uniform(y[0], y[1])
        self.controller.step(action="Teleport", position=dict(random_x, random_y, 0), standing=True)


    def compute_distance(self, ag_pos):
        x = abs(ag_pos['x']-self.obj_position['x'])
        z = abs(ag_pos['z']-self.obj_position['z'])
        return np.sqrt(x**2+z**2)
        
    def domain_rand(self):
        if self.ttv=='train':
            event = self.controller.step(
                action="RandomizeMaterials",
                useTrainMaterials=True,
                useValMaterials=False,
                useTestMaterials=False
            )
            obj_vect = self.choose_object(event)
            return event, obj_vect
        elif self.ttv=='val':
            event = self.controller.step(
                action="RandomizeMaterials",
                useTrainMaterials=False,
                useValMaterials=True,
                useTestMaterials=False
            )
            obj_vect = self.choose_object(event)
            return event, obj_vect
        elif self.ttv=='test':
            event = self.controller.step(
                action="RandomizeMaterials",
                useTrainMaterials=False,
                useValMaterials=False,
                useTestMaterials=True
            )
            obj_vect = self.choose_object(event)
            return event, obj_vect
        
    def choose_object(self, event):
        total_objects = len(event.metadata["objects"])
        while True:
          ix = np.random.randint(0, total_objects)
          self.current_obj = event.metadata["objects"][ix]
          self.obj_position = self.current_obj["position"]
          if self.current_obj["objectType"] in objects and self.current_obj["objectType"] not in self.chosen_objects:
            break;
        self.chosen_objects.append(self.current_obj["objectType"])
        self.obj_vect = self.text_processor(objects[self.current_obj["objectType"]])
        return self.obj_vect
        

In [ ]:
# evaluate policy own helper function bc stable baselines goofy ahhhhhhhhhhh

def evaluate(env, model):
    episode_rewards = []
    lstm_state = []
    
    for _ in range(10):
      obs = env.reset()
      step_rewards = []
      
      while True:
        action, _states = model.predict(obs, deterministic=True)
        #print("states: ", _states[0].shape, _states[1].shape)
        print("action:", action)
        print("proba: ", predict_proba(model, obs, _states)[0].distribution.probs)
        obs, reward, done, info = env.step(action)
        step_rewards.append(reward)
        lstm_state = _states
        if done:
          break
      episode_rewards.append(sum(step_rewards))
    return episode_rewards

In [ ]:
from stable_baselines3 import PPO, A2C, DDPG, DQN
from sb3_contrib import RecurrentPPO
import numpy as np

env = AI2thor('train')
print(env.num_objs)

policy_kwargs = dict(
    features_extractor_class = FeatureExtractor
)
model = RecurrentPPO(policy="MlpLstmPolicy", 
            env=env, 
            learning_rate=0.001, 
            verbose=1, 
            device=device, 
            n_steps=32,
            batch_size=32,
            ent_coef=0.35,
            policy_kwargs=policy_kwargs,
            normalize_advantage=True
          )
# 0.1425+0.1428+0.1426+0.1431+0.1430+0.143+0.1429

0
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.load('./model.pt')

In [ ]:
model.learn(250, reset_num_timesteps=False)
model.save("./modelnew.pt")

  warnings.warn(

  warnings.warn(msg)



reset
step:  0
action:  MoveAhead
error:  
step:  1
action:  Done
Finished! End reward:  0.36246091584132645


error:  


  warnings.warn(

  warnings.warn(msg)



reset
step:  0
action:  MoveLeft
error:  
step:  1
action:  MoveAhead
error:  
step:  2
action:  MoveLeft
error:  
step:  3
action:  Done
Finished! End reward:  0.16049924451374287


error:  


  warnings.warn(

  warnings.warn(msg)



...

reset
step:  0
action:  MoveLeft
error:  
step:  1
action:  MoveRight
error:  
step:  2
action:  Done
Finished! End reward:  0


error:  


  warnings.warn(

  warnings.warn(msg)



reset
step:  0
action:  RotateRight
error:  
step:  1
action:  MoveRight
error:  
step:  2
action:  MoveLeft
error:  
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.31        |
|    ep_rew_mean          | 0.361       |
| time/                   |             |
|    fps                  | 0           |
|    iterations           | 7           |
|    time_elapsed         | 1149        |
|    total_timesteps      | 224         |
| train/                  |             |
|    approx_kl            | 0.032874696 |
|    clip_fraction        | 0.237       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.92       |
|    explained_variance   | -0.0581     |
|    learning_rate        | 0.001       |
|    loss                 | -0.743      |
|    n_updates            | 60          |
|    policy_gradient_loss | -0.0432     |
|    value_loss           | 0.00388     |
-----------------------------------------


  warnings.warn(

  warnings.warn(msg)



reset
step:  0
action:  MoveLeft
error:  
step:  1
action:  MoveRight
error:  
step:  2
action:  MoveRight
error:  
step:  3
action:  MoveAhead
error:  
step:  4
action:  Done
Finished! End reward:  0.5870169931432672


error:  


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

n_eval_episodes = 5

outputs = []


model.load('./modelnew.pt')

test_env = AI2thor('test')

for x in range(1):
  print(x+1)
  x = evaluate_policy(model, test_env, n_eval_episodes=5, deterministic=False)
  print("complete")
  outputs.append(x)

print(outputs)

  warnings.warn(



1


  warnings.warn(

  warnings.warn(msg)



reset
step:  0
action:  MoveLeft
error:  
step:  1
action:  MoveLeft
error:  FP326:StandardDoor1.019 is blocking Agent 0 from moving by (0.0000, 0.0000, 0.2500).
step:  2
action:  MoveAhead
error:  
step:  3
action:  RotateLeft
error:  
step:  4
action:  Done
Finished! End reward:  0.10501786539978997


error:  


  warnings.warn(

  warnings.warn(msg)



...

  warnings.warn(

  warnings.warn(msg)



reset
step:  0
action:  MoveAhead
error:  
step:  1
action:  MoveBack
error:  
step:  2
action:  MoveBack
error:  
step:  3
action:  MoveLeft
error:  
step:  4
action:  MoveRight
error:  
step:  5
action:  RotateRight
error:  
step:  6
action:  MoveRight
error:  StoveBase2 is blocking Agent 0 from moving by (0.2500, 0.0000, 0.0000).
step:  7
action:  MoveBack
error:  
step:  8
action:  RotateRight
error:  
step:  9
action:  RotateRight
error:  
step:  10
action:  MoveBack
error:  
step:  11
action:  MoveAhead
error:  
step:  12
action:  MoveLeft
error:  OVENDOORBOTTOM.001 is blocking Agent 0 from moving by (0.2500, 0.0000, 0.0000).
step:  13
action:  MoveBack
error:  
step:  14
action:  RotateRight
error:  
step:  15
action:  Done
Finished! End reward:  0.5810322329967559


error:  


  warnings.warn(

  warnings.warn(msg)



reset
step:  0
action:  MoveLeft
error:  
step:  1
action:  MoveBack
error:  


KeyboardInterrupt: ignored

In [ ]:
test_env = AI2thor('test')

#model.load("./modelnew.pt")

for x in range(1):
  print(x+1)
  x = evaluate(test_env, model)
  print("complete")
  outputs.append(x)

print(outputs)

NameError: ignored